# Signature Elgamal

On choisit p un nombre premier et un générateur sur $Z/pZ$ g = 3.
- Alice a pour clef privée a et clef publique A = g^a mod p
- Bob a pour clef privée b et clef publique B = g^b mod p

1. Alice envoie A à Bob
2. Bob envoie B à Alice
3. $K = A^b = B^a = g^{ab} \quad \text{mod p}$ est la clef de chiffrement / secret partagé

In [2]:
import sys
sys.path.append("rendu")
import common
from random import randint

In [ ]:
def gen_elgamal_pg(n):
    p = common.gen_prime(n)
    return [p, 3]

In [ ]:
def gen_elgamal_sk_pk(p,g):
    """
    entree : p premier, g générateur sur Z/pZ
    sortie : sk clé privée, pk clé publique
    """
    sk = randint(3, p-2)
    pk = common.expo_modulaire_fast(sk, g, p) # g^sk mod p
    return [sk, pk]

In [ ]:

def gen_elgamal_get_secret(pk_a,sk_a,pk_b,sk_b,p):
    """ 
    entree : (pk_a,sk_a) clé publique et privée de A, (pk_b,sk_b) clé publique et privée de B, p premier
    sortie : secret partagé, clef secrète commune
    """
    secret_1 = common.expo_modulaire_fast(sk_a, pk_b, p)
    secret_2 = common.expo_modulaire_fast(sk_b, pk_a, p)
    assert(secret_1 == secret_2)
    return secret_1

Une fois le secret partagé $K = g^{a,b}$, pour chiffrer un message m, elle calcule $c = mK \text{ mod p}$ et pour déchiffrer $m = cK^{-1} \text{ mod p}$

In [4]:
def enc_elgamal(m,secret,p):
    """
    entree : m message, pk clé publique, p premier
    sortie : (c1,c2) message chiffré
    """
    int_m = common.string_to_int(m)
    return common.expo_modulaire_fast(1, int_m*secret, p) # m*secret mod p
def dec_elgamal(c, secret, p):
    """
    entree : c message chiffré, secret clé secrète, p premier
    sortie : m = c1^secret^-1 mod p
    """
    k = common.inverse_modulaire(secret, p)
    return common.int_to_string(common.expo_modulaire_fast(1, c*k, p))


Pour signer un message m, avec elgamal Alice (de clef privée a) effectue la procédure suivante:
1. Alice choisit un nombre aléatoire $ 1 < k < p-1$
2. Calculer $r = g^k \text{ mod p}$
3. Calculer $s = (m - a r)k^{-1} \text{ mod p-1}$
4. La signature est $(r,s)$

In [ ]:
   
def elgamalsignature(g,p,sk,m):
    """ 
    entree : g générateur sur Z/pZ, p premier, sk clé privée, m message
    sortie : [r,s] signature tel que r = g^k mod p, s = (m - sk*r)k^-1 mod (p-1)
    """
    print("")
    ke = 2
    while common.pgcd(ke,p-1) != 1:
        ke = randint(2,p-2)
    m_int = common.str_to_int(m)
    r = common.expo_modulaire_fast(ke, g, p)
    inv_ke = common.inverse_modulaire(ke, p)
    s = common.expo_modulaire_fast(1, (m_int - sk *r)*inv_ke, p-1)
    return [r,s]

Alice de clef publique $A=g^a$ envoie le message m et la signature $(r,s)$ à Bob
Pour vérifier la signature elgamal, Bob effectue la procédure suivante : 
1. Vérifier que $1 < r < p-1$ et $1 < s < p-1$ sinon la signature est invalide
2. Calcule de $v_1 = A^r r^s \text{ mod p}$
3. Calcule de $v_2 = g^m \text{ mod p}$
4. La signature est valide si $v_1 = v_2$


In [ ]:

def elgamalverification(g,p,r,s,m,pk):
    """ 
    entree : g générateur sur Z/pZ, p premier, r,s signature, m message, pk clé publique (g^sk mod p)
    sortie : booléen vérifiant la signature
    """
    m_int = common.str_to_int(m)
    y = common.expo_modulaire_fast(r, pk, p) # pk^r mod p
    z = common.expo_modulaire_fast(s, r, p) # r^s mod p
    t = common.expo_modulaire_fast(1, y*z, p) # y*z mod p
    alpha_x = common.expo_modulaire_fast(m_int, g, p) # g^m mod p
    return t == alpha_x 